<a href="https://colab.research.google.com/github/johnnystrada/dataprojects/blob/master/mohs_hardness_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:

# Read in the data
train = pd.read_csv('/train.csv')

# create a dataframe with the independent variables
X = train.drop(columns=['id', 'Hardness'])

# create a dataframe with the dependent variable
y = train['Hardness']

model = DecisionTreeRegressor()

# run a gridsearch cv and find the best parameters for the model
param_grid = {'max_depth': [4, 5, 6, 7, 8, 9, 10],
              'min_samples_split': [2, 3, 4, 5, 6,],
              'min_weight_fraction_leaf': [0.01, 0.02, 0.03, 0.04, 0.05],
              'min_impurity_decrease': [0.0, 0.01, 0.02, 0.03, 0.04, 0.05]}
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X, y)
best_params = grid_search.best_params_
# assign the best parameters to the model
model = DecisionTreeRegressor(max_depth=best_params['max_depth'],
                              min_samples_split=best_params['min_samples_split'],
                              min_weight_fraction_leaf=best_params['min_weight_fraction_leaf'],
                              min_impurity_decrease=best_params['min_impurity_decrease'])

# fit the model
model.fit(X, y)

# load in the test data
test = pd.read_csv('/test.csv')
test_id = pd.DataFrame(test['id'])
# drop the id column
test = test.drop(columns=['id'])
# apply the model to the test data
y_pred = model.predict(test)
test = pd.DataFrame(y_pred)
test = test.join(pd.DataFrame(test_id.id))
test = test.rename(columns={0: 'Hardness'})
submission = pd.DataFrame({ 'id': test.id.values, 'Hardness': test.Hardness.values })
submission.to_csv('submission.csv', index=False)